# NestedFrame `map_rows` Demo

In [4]:
from nested_pandas.datasets import generate_data
import numpy as np

In [33]:
nf = generate_data(3,10, seed=1)
nf

a         b                                             nested
0  0.417022  0.604665  [{t: 3.725204, flux: 68.650093, band: 'r'}; …]...
1  0.720324  0.293512  [{t: 6.911215, flux: 83.462567, band: 'g'}; …]...
2  0.000114  0.184677  [{t: 7.935349, flux: 1.828828, band: 'r'}; …] ...

## The "default" example -- dictionary packaging

In [34]:
# row is a dict with keys corresponding to the rows of the nestedframe
def example_func(row):
    return np.mean(row["nested.t"]), np.mean(row["nested.t"]) - row["a"]

nf.map_rows(example_func, output_names=["mean", "mean_minus_base"])

,mean,mean_minus_base
0,8.511203,8.094181
1,9.498250,8.777925
2,10.795447,10.795333


## Column Selection

In [35]:
# setting columns sub-selects which columns are packaged into the dict
nf.map_rows(example_func, columns=["a", "nested.t"], output_names=["mean", "mean_minus_base"])

,mean,mean_minus_base
0,8.511203,8.094181
1,9.498250,8.777925
2,10.795447,10.795333


### Performance Impacts of Column Selection

In [36]:
%%timeit
nf.map_rows(example_func, output_names=["mean", "mean_minus_base"])

789 μs ± 24.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [37]:
%%timeit
nf.map_rows(example_func, columns=["a", "nested.t"], output_names=["mean", "mean_minus_base"])

281 μs ± 21.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Selecting one Nested Structure

In [38]:
# Write a function that programmatically returns the first value from each column in the row
def first_val(row):
    return {"first_"+key.split(".")[1]:row[key][0] for key in row.keys()}

# Can select all nested subcolumns by just passing the name of the nested column
nf.map_rows(first_val, columns="nested")

,first_t,first_flux,first_band
0,3.725204,68.650093,r
1,6.911215,83.462567,g
2,7.935349,1.828828,r


## `args`-based inputs

In [39]:
# Write a function that takes in individual arguments
def example_func(a, time):
    return np.mean(time), np.mean(time) - a

# Setting row_container="args" passes the inputs as individual arguments
nf.map_rows(example_func,
            columns=["a", "nested.t"],
            output_names=["mean", "mean_minus_base"],
            row_container="args")

,mean,mean_minus_base
0,8.511203,8.094181
1,9.498250,8.777925
2,10.795447,10.795333


## Non-df inputs

In [40]:
def example_func(row, scale):
    return np.mean(row["nested.t"]) * scale

# Arguments that are not tied to a column can be passed separately
nf.map_rows(example_func, columns=["nested.t"], output_names="mean", scale=20)

,mean
0,170.224051
1,189.964999
2,215.908938


## Returning Nested Structures

In [42]:
def example_func(row):
    '''map_rows will return a NestedFrame with nested structure'''
    return row["nested.t"] - row["a"], row["nested.t"] - row["b"]

# behavior controlled by "infer_nesting" kwarg, which is true by default
nf.map_rows(example_func, output_names=["offsets.t_a", "offsets.t_b"])

offsets
0  [{t_a: 3.308182, t_b: 3.120539}; …] (10 rows)
1   [{t_a: 6.19089, t_b: 6.617703}; …] (10 rows)
2  [{t_a: 7.935235, t_b: 7.750672}; …] (10 rows)